In [2]:
%matplotlib inline
import time
import random
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from numpy import *
import matplotlib as mpl
import matplotlib.cm as cm
from sympy import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import os, sys
from scipy.optimize import OptimizeWarning
import seaborn as sns; import pandas as pd
plt.rcParams.update({'font.size': 14})

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def calcvelocity(rrange,sigma,vdiff,A,vmin):
    sigmaterm = sigma**2
    distances = np.linspace(0,rrange,100)
    I = A*np.exp(-distances**2/sigmaterm)
    return distances, vdiff*np.exp(-I/A)+vmin

def tothecenter(xpos,ypos,angleh):
    value = math.atan(ypos/xpos)*180/math.pi
    if(xpos < 0): angleh = value+180
    else: angleh = value
    if(angleh < 0): angleh += 360
    if(angleh > 360): angleh -= 360
    return angleh, 2

def reflect(xpos,ypos,dirangle):
    value = math.atan(ypos/xpos)*180/np.pi
    angles = np.zeros(3); checks = []
    if(xpos < 0): angles[0] = value+180
    else: angles[0] = value
    if(angles[0] < 0): angles[0] += 360
    if(angles[0] > 360): angles[0] -= 360
    for i in range(1,3): angles[i] = angles[0] + 360 * pow(-1,i)
    for i in range(3): checks.append(abs(angles[i] - dirangle))
    newangle = dirangle + 2 * (angles[checks.index(min(checks))] - dirangle) + 180
    if (newangle > 360): newangle -= 360       
    return newangle, 1, value

def reflectrand(xpos,ypos,angleh):
    value = math.atan(ypos/xpos)*180/math.pi
    if(xpos < 0): angleh = value+180
    else: angleh = value
    angleh += np.random.uniform(-45,45)
    if(angleh < 0): angleh += 360
    if(angleh > 360): angleh -= 360
    return angleh, 1, value
    
def linear(x, m, n): return m*x+n

def disk_line_picking(s,Rin):
    return 4*s/(np.pi*Rin**2)*np.arccos(s/(2*Rin))-2*s**2/(np.pi*Rin**3)*np.sqrt(1-s**2/(4*Rin**2))
    
def calcintensity(laser,laserframe,delay):
    m = -5/4#-5/2 #-7/5
    n = 9/8#4/3#7/4  #1
    if delay >= 0:   #positive delay - past   - calc velocity due to sensed intensity[i-delay]
        if laserframe < delay: laser[laserframe,j,4] = m*laser[0,j,3]+n   #if i < delay
        else: laser[laserframe,j,4] = m*laser[laserframe,j,3]+n
    else:            #negative delay - future
        if laserframe >= fitlength:    #extrapolate intensity value
            popt,pcov = curve_fit(linear, fitwave, laser[laserframe-fitlength:laserframe,j,3],p0 = (0,0))
            futurevalue = linear(-delay,*popt)
            vvalue = m*futurevalue+n
            laser[laserframe,j,4] = vvalue
        else:     # i < delay
            if laserframe < 2: laser[laserframe,j,4] = m*laser[laserframe,j,3]+n
            else:
                popt,pcov = curve_fit(linear, fitwave[fitlength-laserframe:fitlength], laser[0:laserframe,j,3],p0 = (0,0))
                futurevalue = linear(-delay,*popt)
                laser[laserframe,j,4] = futurevalue
    if laser[laserframe,j,4] > 1: laser[laserframe,j,4] = 1
    if laser[laserframe,j,4] < 0.1: laser[laserframe,j,4] = 0
    return laser

def calcintensity2(laser,laserframe,delay):
    if delay >= 0:   #positive delay - past   - calc velocity due to sensed intensity[i-delay]
        if laserframe < delay: laser[laserframe,j,4] = vfactor*math.exp(-2*laser[0,j,3]/amplitude)+vmin   #if i < delay
        else: laser[laserframe,j,4] = vfactor*math.exp(-2*laser[laserframe-delay,j,3]/amplitude)+vmin
    else:            #negative delay - future
        if laserframe >= fitlength:    #extrapolate intensity value
            popt,pcov = curve_fit(linear, fitwave, laser[laserframe-fitlength:laserframe,j,3],p0 = (0,0))
            futurevalue = linear(-delay,*popt)
            vvalue = vfactor*math.exp(-2*futurevalue/amplitude)+vmin
            if vvalue >= 0 and vvalue <=1: laser[laserframe,j,4] = vvalue
            else:                     # 0 < intensity < 1
                if vvalue > 1: laser[laserframe,j,4] = 1
                if vvalue < 0: laser[laserframe,j,4] = 0 
        else:     # i < delay
            if laserframe < 2: laser[laserframe,j,4] = vfactor*math.exp(-2*laser[laserframe,j,3]/amplitude)+vmin
            else:
                popt,pcov = curve_fit(linear, fitwave[fitlength-laserframe:fitlength], laser[0:laserframe,j,3],p0 = (0,0))
                futurevalue = linear(-delay,*popt)
                if vvalue >= 0 and vvalue <= 1: laser[laserframe,j,4] = vvalue
                else:
                    if vvalue > 1: laser[laserframe,j,4] = 1
                    if vvalue < 0: laser[laserframe,j,4] = 0  
    return laser

def vectorlength(x): return np.sqrt(pow(x[0],2)+pow(x[1],2))

def calcdrifttheory(v,r,tau,delay,base):
    dv = np.gradient(v,base)
    return tau/2*(1-delay/tau)*v*dv+tau*v**2/r

def checkintersection(particle,connections): #collision of particles
    check = False
    while check == False:
        part1 = 0; part2 = 1; counter=0
        for k in range(connections):
            x1=particle[i,part1,0];y1=particle[i,part1,1]
            x2=particle[i,part2,0];y2=particle[i,part2,1]
            distance=math.sqrt(pow(x1-x2,2)+pow(y1-y2,2))
            if distance<diameter*0.99:
                counter+=1
                intersection=diameter-distance
                xdist=(x1-x2)/distance*intersection/2*(1+faster*counter)
                ydist=(y1-y2)/distance*intersection/2*(1+faster*counter)
                particle[i,part1,0]+=xdist
                particle[i,part2,0]-=xdist
                particle[i,part1,1]+=ydist
                particle[i,part2,1]-=ydist
                x1=particle[i,part1,0];y1=particle[i,part1,1]
                x2=particle[i,part2,0];y2=particle[i,part2,1]
                distance=math.sqrt((x1-x2)**2+(y1-y2)**2)
                if distance<diameter*0.99: 
                    connections -= 1
                    counter += 1
                    k -= 1
            else: 
                part2 += 1
                if part2 == pnumber:
                    part1 += 1
                    part2 = part1 + 1
        if counter == 0: check=True
        if counter > accuracy: check=True
    return particle

def calcprofile(points,m,n):
    distmum = np.linspace(0,D_arena*1e-6,points)
    intensity = np.zeros(points)
    intensity = 2*np.exp(-distmum**2/sigma**2)/2
    distint = np.linspace(0,1,points)
    velocitysim = np.zeros(points)
    for i in range(points):
        value = m*distint[i]+n
        if value < 0: value = 0
        if value > 1: value = 1
        velocitysim[i] = value
    velocitydist = np.zeros(points)
    for i in range(points):
        check = True; count2 = 0
        while check == True:
            if distint[count2] >= intensity[i]:
                velocitydist[i] = velocitysim[count2]
                check = False
            else: count2 += 1        
    smoothvalue = 50
    test = np.gradient(velocitydist)/(distmum[1]-distmum[0])
    diffv = smooth(test,smoothvalue)
    maxpos = np.argmax(diffv)
    maxdist = distmum[maxpos]
    ddnew = np.linspace(0,boundaryradius/maxdist,points)
    return distmum*1e6, ddnew, velocitydist, maxdist, maxpos, diffv

In [3]:
dt = 0.05
howlong = 36
alldata = np.zeros((howlong,3))
count1=1;count2=0
tauwave = [0.4,0.8,1.2,1.6]
for i in range(howlong):
    alldata[i,0] = count1
    alldata[i,1] = np.sqrt(240/(tauwave[count2]-dt))
    alldata[i,2] = i
    count1 += 4
    if count1 > 34:
        print(round(alldata[i,1],2))
        count1 = 1
        count2 += 1
        
print(alldata)

26.19
17.89
14.45
12.44
[[ 1.         26.18614683  0.        ]
 [ 5.         26.18614683  1.        ]
 [ 9.         26.18614683  2.        ]
 [13.         26.18614683  3.        ]
 [17.         26.18614683  4.        ]
 [21.         26.18614683  5.        ]
 [25.         26.18614683  6.        ]
 [29.         26.18614683  7.        ]
 [33.         26.18614683  8.        ]
 [ 1.         17.88854382  9.        ]
 [ 5.         17.88854382 10.        ]
 [ 9.         17.88854382 11.        ]
 [13.         17.88854382 12.        ]
 [17.         17.88854382 13.        ]
 [21.         17.88854382 14.        ]
 [25.         17.88854382 15.        ]
 [29.         17.88854382 16.        ]
 [33.         17.88854382 17.        ]
 [ 1.         14.44630237 18.        ]
 [ 5.         14.44630237 19.        ]
 [ 9.         14.44630237 20.        ]
 [13.         14.44630237 21.        ]
 [17.         14.44630237 22.        ]
 [21.         14.44630237 23.        ]
 [25.         14.44630237 24.        ]
 

In [4]:
name = 'test'
np.random.seed(4) 

pnumber = 2
dt = 0.0519                                       #iteration time of the camera [s]
stepsexp  = 100000                                    #number of camera pictures
reorientationtimeinput = 1
noise = np.sqrt(240/(reorientationtimeinput-dt))
#noise = 10                                       #Sigma for the Gaussian noise for the angle
delay = 1                                      #Delay, negative value for anticipation
maxvelocityin = 4.5e-6#5.25e-6                           #velocity of the particle depending on the laser power [m/s]
steps = 11                                                #Fig 1c; https://www.nature.com/articles/s41467-018-06445-1.pdf
amplitude = 1
vmin = 0                                     #Min velocity
vmax = 1                                        #Max velocity
pxtomum=0.0533e-6                               #px to µm [µm/px]
sigma = 80*pxtomum                                     #Sigma for the Gaussian intensity profile [px]
D_0 = 3.85e-14                                  #measured diffusion coefficient D_0 = kB*T/(6*np.pi*eta*radius)
kB = 1.38064852e-23                              #Boltzmanns constant [J/K]
T = 273.15 + 20                                  #Room temperature [K]
eta = 2.414e-5*10**(247.8/(T-140))               #Viscosity [Pa*s]
offset = 12                                     #offset of the laser [px]
boundaryradius = pxtomum * 150                  #Radius of the boundary [px]
diameter = 2.19e-6                              #diameter of the particle [m]
R = diameter / 2
fitlength = 5                                   #for negative delay, linear fit on the last N measured intensities to extrapolate intensity
diffusion = True                                #Switching diffusion on and off
savename = str(int(noise))+' - '+str(int(delay))
sigmat = 2 * sigma * sigma
D_arena = 2*boundaryradius*1e6
accuracy=1000
faster=0.1          #0.2 or 0.1 make it faster, 0 exact
connections = int(pnumber*(pnumber-1)/2)
savefolderplus = '.'; savefolder = '.'
shortlength = stepsexp
prefactor = np.sqrt(2*D_0*dt)                 #value for calculation of the random steps
sumhitboundary = 0
fitwave = np.linspace(-fitlength+1,1,fitlength,endpoint=False)
sigmafactor = pow(sigma,2)
vfactor=vmax-vmin
radius = diameter/2
timewave = np.linspace(0,dt*stepsexp,stepsexp,endpoint=False)  #Timewave for sub cam resolution

plothisto = True#False#
plotVelocityvsCoM = True#False

In [7]:
calctime = 0
reorientationtime = meandensity = reorientationtime2 = Drot = Droterr = Drotexp = Drotexperr = np.nan
ptime = ptimeerr = plength = plengtherr = maxvelocity = meanvelocity = centermorein = centermoreout = np.nan
meantime = meanprob = notone = zerocount = meanvelocity = meanchange = np.nan
for p in range(len(alldata)):
#for p in range(25,len(alldata)):
#for p in range(1):
    start_time = time.time() 
    noise = int(alldata[p,1])                             #Sigma for the Gaussian noise for the angle
    delay = int(alldata[p,0])                             #Delay, negative value for anticipation
    name = str(int(noise)) + ' - ' + str(int(delay))
    
    laser = np.zeros((shortlength,pnumber,5))              #laserposition  0 - x; 1 - y; 2 - time at which the laser is set to the new position
                                                 #               3 - Ext Int it measures; 4 - theo speed(Int laser)
    particle = np.zeros((stepsexp,pnumber,8))    #cam detection  0 - x; 1 - y; 2 - time of the average particle position
                                                     #               3 - time at which the the laser is placed 
                                                     #               4 - path number
                                                     #               5 - angle; 6 - out
                                                     #               7 - Distance to center        
    noisesteps = np.zeros((stepsexp,pnumber,2)); anglenoise = np.zeros((stepsexp,pnumber))
    for j in range(pnumber):
        for i in range(2): noisesteps[:,j,i] = prefactor*np.random.normal(0,1,stepsexp) #Create random steps
    if noise != 0: 
        for j in range(pnumber): anglenoise[:,j] = noise*np.random.normal(0,1,stepsexp)           #Create random angle noise
    deltas = np.zeros(2)
    rnoise = np.sqrt(np.random.uniform(0,boundaryradius**2,10*pnumber))
    anglenoise2 = np.random.uniform(0,2*np.pi,10*pnumber)
    anglenoisecos = np.cos(anglenoise2)
    anglenoisesin = np.sin(anglenoise2)
    smaller = pow(diameter,2)
    count = 0
    startx = np.zeros(pnumber); starty = np.zeros(pnumber)
    for j in range(pnumber):
        check = False
        while check == False:
            startx[j] = rnoise[count] * anglenoisecos[count]
            starty[j] = rnoise[count] * anglenoisesin[count]
            if j >= 1:
                pardist = np.zeros(j)
                for k in range(j): pardist[k] = pow(startx[j]-startx[k],2)+pow(starty[j]-starty[k],2)
                if min(pardist) >= smaller: check = True
            else: check = True
            count += 1
    startangle = np.random.uniform(0,360,pnumber)
    particle[0,:,0] = startx; particle[0,:,1] = starty
    particle[0,:,5] = startangle
            
    alldist = np.zeros((stepsexp,connections))
    out = np.zeros(pnumber)
    startpoint = [0]; endpoint = []
    particle[:,:,4] = maxvelocityin
    for i in range(1,stepsexp):
        particle[i,:,2] = particle[i-1,:,2] + dt                
        particle[i,:,:2] = particle[i-1,:,:2]                              #Position equals position before
        particle[i,:,5] = particle[i-1,:,5]
        particle[i,:,7] = np.sqrt(pow(particle[i,:,0],2) + pow(particle[i,:,1],2))   #distance to the center
        for j in range(pnumber):
            if particle[i,j,7] > boundaryradius:      #particle is out
                if particle[i,j,7] > 1.1 * boundaryradius: particle[i,j,5], out = tothecenter(particle[i,j,0],particle[i,j,1],particle[i,j,5])  #sendback to the center if something really fails
                else:
                    if out == 0:   #reflect randomly at the boundary
                        particle[i,j,5], out, incidentangle = reflectrand(particle[i,j,0],particle[i,j,1],particle[i,j,5])
                        if j == 0: endpoint.append(i)
            else: 
                if particle[i-1,j,6] != 0 and j == 0: startpoint.append(i)
                out = 0
            particle[i,j,6] = out
            if out == 0: particle[i,j,5] += anglenoise[i,j]   #add noise at the angle if it is inside
            if particle[i,j,5] > 360: particle[i,j,5] -= 360
            if particle[i,j,5] < 0: particle[i,j,5] += 360
            laser[i,j,0] = particle[i,j,0]+offset*pxtomum*np.cos(particle[i,j,5]*np.pi/180)   #laser position      
            laser[i,j,1] = particle[i,j,1]+offset*pxtomum*np.sin(particle[i,j,5]*np.pi/180)
            intensity = 0
            for k in range(pnumber):
                if k != j:
                    xdiff = particle[i-delay,j,0]-particle[i-delay,k,0]
                    ydiff = particle[i-delay,j,1]-particle[i-delay,k,1]
                    dist = np.sqrt(pow(xdiff,2)+pow(ydiff,2))-2*R
                    intensity += amplitude*np.exp(-dist*dist/sigmafactor)
            intensity /= pnumber
            if intensity > 1: intensity = 1
            laser[i,j,3] = intensity#amplitude*math.exp(-pow(particle[i,j,7],2)/sigmafactor)        #intensity of the external field
            laser = calcintensity2(laser,i,delay)     #calc laser intensity depending on the field intensity 
            if diffusion == True:
                for k in range(2): particle[i,j,k] += noisesteps[i,j,k]         #add diffusion of the particle
            for k in range(2): deltas[k] = particle[i,j,k]-laser[i,j,k]
            particle[i,j,3] = np.sqrt(pow(deltas[0],2)+pow(deltas[1],2))      #calc particle-laser distance
            particle[i,j,4] = maxvelocityin
            for k in range(2): particle[i,j,k] += particle[i,j,4]/particle[i,j,3]*deltas[k]*dt*laser[i,j,4]  #add thermophoretic movement  
        particle = checkintersection(particle,connections)
        part1 = 0; part2 = 1
        for k in range(connections):
            x1=particle[i,part1,0];y1=particle[i,part1,1]
            x2=particle[i,part2,0];y2=particle[i,part2,1]
            dist=math.sqrt(pow(x1-x2,2)+pow(y1-y2,2))
            alldist[i,k] = dist
            part2 += 1
            if part2 == pnumber:
                part1 += 1
                part2 = part1 + 1  
                
    if particle[i,0,7] < boundaryradius: endpoint.append(i-1)   
    if startpoint[-1] == shortlength: 
        del(startpoint[-1])
        del([endpoint[-1]])
    #------------------------------------------
    maxlength = stepsexp
    distancetocenter=[]                                    #Calc velocity 
    distancetocenter=np.sqrt(pow(particle[:,0,0],2)+pow(particle[:,0,1],2))
    meandistance = np.mean(distancetocenter)
    innerarea = 0
    for i in range(maxlength):
        if distancetocenter[i] < 4*R: innerarea += 1
    innerarea /= maxlength
    timeclosewave = []; count = 0
    for i in range(1,maxlength): 
        if distancetocenter[i] < 4*R: count += 1
        else:
            if distancetocenter[i-1] < 4*R:
                timeclosewave.append(count*dt)
                count = 0
    timeclose = np.mean(timeclosewave)
    string_temp = savefolderplus + '/DataTimeclose/'
    if not os.path.exists(string_temp): os.makedirs(string_temp)
    np.savetxt(string_temp + 'DataTimeclose'+str(p)+'.txt', timeclosewave,fmt='%1.6f')
    
    oldstartpoint = []; oldendpoint =[]
    if distancetocenter[0] < boundaryradius*pxtomum:
        oldstartpoint.append(0)
        check = 0
    else: check = 1; remi = -1   
    count = 0
    for i in range(1,maxlength):
        if distancetocenter[i] > boundaryradius*pxtomum:
            if check == 0:
                remi = i
                check = 1
            count += 1
        if check == 1: 
            if distancetocenter[i] < boundaryradius*pxtomum:
                check = 0
                if remi != -1:
                    oldstartpoint.append(i)
                    oldendpoint.append(remi)
                else: oldstartpoint.append(i)
            else: count += 1   
    if check == 1: oldendpoint.append(remi)
    else: oldendpoint.append(maxlength-1)

    realvelocity = np.zeros(maxlength);stepsize = np.zeros(maxlength)    
    movingvector = np.zeros((maxlength,2)); directionvector = np.zeros((maxlength,2))
    directionvector[:,0] = np.cos(particle[:,0,3]*np.pi/180)
    directionvector[:,1] = np.sin(particle[:,0,3]*np.pi/180)
    for i in range(2,maxlength):                           #Calc projection of the velocity in the required direction
        movingvector[i,0] = particle[i,0,0]-particle[i-1,0,0]
        movingvector[i,1] = particle[i,0,1]-particle[i-1,0,1]
        stepsize[i] = abs(np.dot(directionvector[i-2,:], movingvector[i,:]))
        realvelocity[i] = stepsize[i]/dt

    particlesdisplay = np.zeros((maxlength,pnumber,8))
    particlesdisplay[:,:,:] = particle[:,:,:]
    for i in range(pnumber):
        particlesdisplay[:,i,:2] -= particle[:,1,:2]
        particlesdisplay[:,i,4:6] -= particle[:,1,:2]
        particlesdisplay[:,i,:2] *= 1e6
        particlesdisplay[:,i,4:6] *= 1e6
    #------------------------------------------
    newangle = np.zeros(stepsexp); newangle = particle[:,0,5].copy()    #MSD for the theoretic moving direction
    diff = np.zeros(3)
    for i in range(1,stepsexp):
        diff[0] = abs(newangle[i]-newangle[i-1])
        diff[1] = abs(newangle[i]-newangle[i-1]+360)
        diff[2] = abs(newangle[i]-newangle[i-1]-360)
        mini = min(diff)
        if mini != diff[0]:
            if mini == diff[1]: newangle[i:] += 360
            else: newangle[i:] -= 360
    for i in range(len(endpoint)-1): newangle[endpoint[i]:] -= newangle[endpoint[i]]-newangle[endpoint[i]-1]

    if noise != 0:
        check = True; i = 1
        reorwave= np.zeros((stepsexp,2))
        while check == True:
            diff = np.zeros(stepsexp-i)
            for j in range(i,stepsexp): diff[j-i] = abs(newangle[j] - newangle[j-i])  #Calculate the direction change for i steps
            mean = np.mean(diff[:stepsexp-i])   #Calculate the standard derivation
            reorwave[i,0] = i*dt
            reorwave[i,1] = mean
            if mean/57.3 >= 1: 
                check = False
                reorlen = i
            else: i += 1 
            if i == stepsexp:
                reorlen = i
                i = 666/dt
                check = False       
        reorientationtime = i*dt 
    else: reorientationtime = 666
    
    maxlength = stepsexp
    particles = np.zeros((stepsexp,pnumber,8))
    particles[:,:,0] = particle[:,:,0].copy()
    particles[:,:,1] = particle[:,:,1].copy()
    particles[:,:,2] = particle[:,:,5].copy()
    particles[:,:,0] *= 1e6
    particles[:,:,1] *= 1e6

    directionvector = np.zeros((maxlength,2))
    movingvector = np.zeros((maxlength,2))
    velocity = np.zeros((maxlength,pnumber));stepsize = np.zeros(maxlength)
    velocitywithdiffusion=np.zeros((maxlength,pnumber))
    velocity2 = np.zeros((maxlength,pnumber,2))
    distancetocenter = np.zeros((maxlength,pnumber))

    for j in range(pnumber):
        directionvector[:,0] = np.cos(particlesdisplay[:,j,2]*np.pi/180)
        directionvector[:,1] = np.sin(particlesdisplay[:,j,2]*np.pi/180)
        for i in range(1,maxlength):                           #Calc projection of the velocity in the required direction
            distancetocenter[i,j] = np.sqrt(pow(particles[i,j,0],2)+pow(particles[i,j,1],2))
            movingvector[i,0] = particlesdisplay[i,j,0]-particlesdisplay[i-1,j,0]
            movingvector[i,1] = particlesdisplay[i,j,1]-particlesdisplay[i-1,j,1]
            #stepsize[i] = abs(np.dot(directionvector[i,:], movingvector[i,:]))
            velocitywithdiffusion[i,j] = np.sqrt(pow(movingvector[i,0],2)+pow(movingvector[i,1],2))/dt 
            velocity[i,j] = velocitywithdiffusion[i,j]
            distance = np.sqrt(pow(particles[i,j,0],2)+pow(particles[i,j,1],2))
            if distancetocenter[i,j] < boundaryradius*1e6:
                velocity2[i,j,0] = velocity[i,j]
                velocity2[i,j,1] = velocitywithdiffusion[i,j]
    for j in range(pnumber):
        for i in range(maxlength):
            if velocity2[i,j,0] == 0: velocity2[i,j,:]=np.nan

    string_temp = savefolderplus + '/PlotVelocity/'
    if not os.path.exists(string_temp): os.makedirs(string_temp)
    wave = velocity2[:,:,0]
    binwave=np.linspace(np.amin(wave[~np.isnan(wave)]),min(np.amax(wave[~np.isnan(wave)]),10),20)
    fig=plt.figure(1, figsize = (7,4) )
    plt.subplot(1,1,1)
    for i in range(pnumber):
        wave = velocity2[:,i,0]    
        plt.hist(wave[~np.isnan(wave)], bins=binwave,alpha = 0.2)
        plt.axvline(x=np.nanmean(wave),linewidth=0.5, color = 'k',linestyle='--')
    plt.xlabel('Velocity [$\mu m/s$]')
    plt.ylabel('accurance')
    #plt.text(3,0,'mean step size 1: {:.2f} µm/s'.format(np.mean(velocity)),fontsize=12)
    plt.legend()
    plt.tight_layout() 
    plt.savefig(string_temp + 'Stepsize vs time'+str(name)+'.pdf', format='pdf')
    plt.close(fig)#plt.show() 
    #---------------------------------
    meanintensity = np.nanmean(laser[:,0,3])    
    test = np.zeros(maxlength)
    for i in range(maxlength):
        count = 0
        for j in range(pnumber):
            if particles[i,j,7] > 0.2: count += 1
        if count >= 3: test[i] = 1
    percentagecluster = sum(test)/maxlength
    #----------------------------------
    plotvwave = np.zeros((maxlength,pnumber))
    plotintwave = np.zeros((maxlength,pnumber))
    for i in range(maxlength-1): 
        plotvwave[i,:] = laser[i+1,:,4]
        plotintwave[i,:] = laser[i+1,:,3]
    maxvelocity = np.zeros(pnumber)
    for k in range(pnumber):
        value = plotvwave[:,k]
        steps2 = 6
        speedhisto= np.zeros((steps2,4))    #0 - angle, 1 - velocity, 2 - counts, 3 - variance
        speedcollect=np.zeros((steps2,maxlength))
        for i in range(steps2): speedhisto[i,0] = min(value)+(i+1)/steps2*(1-min(value))
        for i in range(maxlength-1):                      #Statistics over the velocity in required direction vs the theoretical velocity
            if value[i] < speedhisto[0,0]:
                speedhisto[0,1] += velocity[i,k]
                speedcollect[0,int(speedhisto[0,2])] = velocity[i,k]
                speedhisto[0,2] += 1
            else:
                for j in range(1,steps2):
                    if value[i] > speedhisto[j-1,0] and value[i] < speedhisto[j,0]:
                        speedhisto[j,1] += velocity[i,k]
                        speedcollect[j,int(speedhisto[j,2])] = velocity[i,k]
                        speedhisto[j,2] += 1
        speedhisto[:,1] /= speedhisto[:,2]
        for i in range(maxlength-1):
            for j in range(steps2):
                if speedcollect[j,i] == 0: speedcollect[j,i] = np.nan
        maxvelocity[k] = float(speedhisto[-1,1])
    #------------------------------------------
    driftdelay = 1
    distancetocenter=[]                                    #Calc velocity 
    distancetocenter=np.sqrt(pow(particlesdisplay[:,0,0],2)+pow(particlesdisplay[:,0,1],2))
    movingvector = np.zeros((maxlength,2))
    velocity = np.zeros(maxlength)
    velocitywithdiffusion=np.zeros(maxlength)
    for i in range(2,maxlength):                           #Calc projection of the velocity in the required direction
        movingvector[i,0] = particlesdisplay[i,0,0]-particlesdisplay[i-1,0,0]
        movingvector[i,1] = particlesdisplay[i,0,1]-particlesdisplay[i-1,0,1]
    velocitywithdiffusion[:] = np.sqrt(pow(movingvector[:,0],2)+pow(movingvector[:,1],2))/dt 
    velocity[:] = velocitywithdiffusion[:]
    binwave=np.linspace(min(velocity),min(max(velocitywithdiffusion),10),20)
    countin = 0
    for i in range(maxlength):
        if distancetocenter[i] < sigma*pxtomum: countin += 1
    countin /= maxlength
        
    value = particles[:,0,2]
    steps2 = 6
    speedhisto= np.zeros((steps2,4))    #0 - angle, 1 - velocity, 2 - counts, 3 - variance
    speedcollect=np.zeros((steps2,maxlength))
    for i in range(steps2): speedhisto[i,0] = min(value)+(i+1)/steps2*(1-min(value))
    for i in range(maxlength):                      #Statistics over the velocity in required direction vs the theoretical velocity
        if value[i] < speedhisto[0,0]:
            speedhisto[0,1] += velocity[i]
            speedcollect[0,int(speedhisto[0,2])] = velocity[i]
            speedhisto[0,2] += 1
        else:
            for j in range(1,steps2):
                if value[i] > speedhisto[j-1,0] and value[i] < speedhisto[j,0]:
                    speedhisto[j,1] += velocity[i]
                    speedcollect[j,int(speedhisto[j,2])] = velocity[i]
                    speedhisto[j,2] += 1
    speedhisto[:,1] /= speedhisto[:,2]
    maxvelocitymeasured = speedhisto[-1,1]
    
    startpoint = []; endpoint =[]
    if distancetocenter[0] < boundaryradius*pxtomum:
        startpoint.append(0)
        check = 0
    else: check = 1; remi = -1   
    count = 0
    for i in range(1,maxlength):
        if distancetocenter[i] > boundaryradius*pxtomum:
            if check == 0:
                remi = i
                check = 1
            count += 1
        if check == 1: 
            if distancetocenter[i] < boundaryradius*pxtomum:
                check = 0
                if remi != -1:
                    startpoint.append(i)
                    endpoint.append(remi)
                else: startpoint.append(i)
            else: count += 1   
    if check == 1: endpoint.append(remi)
    else: endpoint.append(maxlength-1)
    
    newangle = np.zeros(maxlength); newangle = particles[:,0,3].copy()    #MSD for the theoretic moving direction
    diff = np.zeros(3)
    for i in range(1,maxlength):
        diff[0] = abs(newangle[i]-newangle[i-1])
        diff[1] = abs(newangle[i]-newangle[i-1]+360)
        diff[2] = abs(newangle[i]-newangle[i-1]-360)
        mini = min(diff)
        if mini != diff[0]:
            if mini == diff[1]: newangle[i:] += 360
            else: newangle[i:] -= 360
    for i in range(len(endpoint)-1): newangle[endpoint[i]:] -= newangle[endpoint[i]]-newangle[endpoint[i]-1]

    radialdata = np.zeros((steps,15))  #0 - bins, 1 - counts, 2 - v, 3 - verr, 4 - v outwards, 5 - v outwards err
    base=(2*boundaryradius-diameter)*1e6/(steps-1)      # 6 - v inwards, 7 - v inwards err, 8 - density, 9 - v difference, 10 - v diff err, 11 - drift, 12 - drift err
    for i in range(steps): radialdata[i,0] = diameter*1e6+i*base
    averagebinposition = np.zeros(steps-1)
    for i in range(1,steps): averagebinposition[i-1] = (radialdata[i,0]+radialdata[i-1,0])/2

    step = np.zeros(maxlength); shell = np.zeros(maxlength); inout = []
    collectdrift = np.zeros((maxlength,steps))
    collectvelocity = np.zeros((maxlength,steps,3))
    count = np.zeros((steps,3))
    radialdrift = 0
    for i in range(1,maxlength):
        j = 0
        check = False
        while check == False:
            if distancetocenter[i] < radialdata[j,0] and distancetocenter[i] > radialdata[j-1,0]:
                check = True
                radialdata[j,1] += 1
                collectvelocity[int(count[j,0]),j,0] = velocity[i]
                count[j,0] += 1                        
                radialdrift = distancetocenter[i]-distancetocenter[i-driftdelay]
                collectdrift[int(count[j,0]),j] = radialdrift
                step[i] = i; shell[i] = round(averagebinposition[j-1],2)
                if radialdrift > 0:  #moving outwards
                    collectvelocity[int(count[j,1]),j,1] = velocity[i]
                    count[j,1] += 1
                    inout.append('outwards')
                if radialdrift < 0:  #moving inwards
                    collectvelocity[int(count[j,2]),j,2] = velocity[i]
                    count[j,2] += 1
                    inout.append('inwards')
            else: j += 1
            if j == steps: check = True
        if shell[i] == 0:
            step[i] = i; shell[i] = 888
            if radialdrift < 0: inout.append('inwards')
            else: inout.append('outwards')

    for i in range(1,steps): 
        radialdata[i,2] = np.mean(collectvelocity[:int(count[i,0]),i,0])    #total velocity
        radialdata[i,3] = np.nanvar(collectvelocity[:int(count[i,0]),i,0])
        radialdata[i,4] = np.mean(collectvelocity[:int(count[i,1]),i,1])    #moving outwards
        radialdata[i,5] = np.nanvar(collectvelocity[:int(count[i,1]),i,1])
        radialdata[i,6] = np.mean(collectvelocity[:int(count[i,2]),i,2])    #moving inwards
        radialdata[i,7] = np.nanvar(collectvelocity[:int(count[i,2]),i,2])
        radialdata[i,8] = radialdata[i,1]/averagebinposition[i-1]                   #density
        radialdata[i,11] = -np.mean(collectdrift[:int(count[i,0]),i])/dt
        radialdata[i,12] = np.nanvar(collectdrift[:int(count[i,0]),i])/dt
    drexp = radialdata[1,0]
    sumdensity = sum(radialdata[1:,8]*drexp)
    radialdata[1:,8] /= sumdensity                                                  
    radialdata[1:,9] = radialdata[1:,6] - radialdata[1:,4]                  #velocity difference
    radialdata[1:,10] = np.sqrt(radialdata[1:,7]**2 + radialdata[1:,5]**2)
    meandensity = sum(averagebinposition*radialdata[1:,8])
    radialdata[1:,13] = count[1:,2]*radialdata[1:,6] - count[1:,1]*radialdata[1:,4]
    radialdata[1:,14] = count[1:,2]/(count[1:,1]+count[1:,2])*100

    distmum, ddnew, velocitydist, maxdist, maxpos, diffv = calcprofile(1000,-5/4,9/8)
    test = np.gradient(velocitydist)/(distmum[1]-distmum[0])
    smoothvalue = 50
    diffv = smooth(test,smoothvalue)
    #diffv = np.gradient(velocitydist)/(distmum[1]-distmum[0])
    maxpos = np.argmax(diffv)
    maxdist = distmum[maxpos]
    sigmawave = np.zeros(steps)
    if maxpos == 0: maxdist = boundaryradius
    #base=boundaryradius*1e6/maxdist/(steps-1)      # 6 - v inwards, 7 - v inwards err, 8 - density, 9 - v difference, 10 - v diff err, 11 - drift, 12 - drift err
    #for i in range(steps): sigmawave[i] = i*base
    #averagesigmaposition = np.zeros(steps-1)
    #for i in range(1,steps): averagesigmaposition[i-1] = (sigmawave[i]+sigmawave[i-1])/2  
    schnitzer = 1/radialdata[1:,2]
    schnitzer /= np.nansum(schnitzer*drexp)
    for i in range(len(schnitzer)):
        if radialdata[1+i,2] == 0: schnitzer[i] = np.nan

    dr = distmum[1]*1e6   
    string_temp = savefolder + '/PlotRadial/'
    if not os.path.exists(string_temp): os.makedirs(string_temp) 
    fig=plt.figure(1, figsize = (15,15) )
    ax1 = plt.subplot(3,2,1)
    color = 'tab:red'
    ax1.set_xlabel(r'r/$\sigma$')
    ax1.set_ylabel('Density [normalized]', color=color)
    bars = ax1.bar(averagebinposition, radialdata[1:,8],base, color=color,alpha = 0.5, edgecolor = color,label='Simulation')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_ylim([0,1.1*max(radialdata[1:,8])])
    ax1.set_xlim([0,2*boundaryradius*1e6])
    ax1.axvline(x=2*R*1e6,linewidth=0.5, color = 'k',linestyle='--')
    #ax1.axvline(x=meandensity,linewidth=0.5, color = 'k',linestyle='--',label = 'mean value')
    ax1.set_title(' ')
    theory = 1/(velocitydist**(1+delay*dt/reorientationtime))
    theory /= np.nansum(theory*dr)             
    line1 = ax1.plot(distmum,theory*1e6, color='red', linewidth = 3,label = 'Volpe theory')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('1/velocity [normalized]', color=color)  # we already handled the x-label with ax1
    line2 = ax2.plot(averagebinposition,schnitzer, color=color, linewidth = 3,label='Schnitzer theory')
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([0,1.1*max(radialdata[1:,8])])
    lns = line1+line2
    labs = [l.get_label() for l in lns]
    ax1.legend(lns,labs)

    dd, vcalc = calcvelocity((2*boundaryradius-2*R),sigma,vfactor,amplitude,vmin)
    ddnew = np.zeros(len(dd))
    ddnew[:] = (dd[:] + 2*R)*1e6
    
    ax3 = plt.subplot(3,2,2)
    ax3.set_ylabel('$v_{in}-v_{out}$ [µm/s]')
    ax3.set_xlabel(r'r [µm]')
    ax3.set_title(' ')
    ax3.scatter(averagebinposition,radialdata[1:,9],label ='$v_{in} - v_{out}$')
    test = np.gradient(vcalc)/(dd[1]-dd[0])
    smoothvalue = 30
    #test2 = smooth(test,smoothvalue)
    plt.plot(ddnew,test/np.nanmax(test)*np.nanmax(radialdata[1:,9]),color='r',label = 'dv/dt')
   
    ax3.axhline(y=0,linewidth=0.5, color = 'k',linestyle='--')
    ax3.axvline(x=2*R*1e6,linewidth=0.5, color = 'k',linestyle='--')
    ax3.set_xlim([0,2*boundaryradius*1e6])
    ax3.legend()

    #distmum, ddnew, velocitydist = calcprofile(200,-5/4,9/8)
    ax4 = plt.subplot(3,2,3)
    ax4.scatter(averagebinposition, radialdata[1:,2],label='measured')
    ax4.axvline(x=2*R*1e6,linewidth=0.5, color = 'k',linestyle='--')
    #ax4.plot(distmum,2*velocitydist*maxvelocitymeasured,label='measured',color='orange')
    ax4.plot(ddnew,vcalc*np.mean(radialdata[-3:,2]),label='measured',color='orange')
    ax4.set_xlabel(r'r [µm]')
    ax4.set_ylabel('Velocity [µm/s]')
    ax4.set_xlim([0,2*boundaryradius*1e6])
    ax4.set_ylim([0,1.1*np.nanmax(radialdata[:,2])])
    ax4.legend()

    ax5 = plt.subplot(3,2,4)
    plt.scatter(averagebinposition,radialdata[1:,11],label='measured')
    ax5.axhline(y=0,linewidth=0.5, color = 'k',linestyle='--')
    ax5.axvline(x=2*R*1e6,linewidth=0.5, color = 'k',linestyle='--')
    drifttheory = calcdrifttheory(velocitydist,distmum,reorientationtime,delay*dt,dr)
    plt.plot(distmum,drifttheory/drifttheory[2]*radialdata[1,11],'k--',label='theory [a.u.]')
    ax5.set_xlabel(r'r/$\sigma$')
    ax5.set_ylabel('Drift [µm/s]')
    ax5.set_xlim([0,2*boundaryradius*1e6])
    ax5.legend()

    percentage2 = count[:,2]/(count[:,1]+count[:,2])*100
    ax6 = plt.subplot(3,2,5)
    plt.plot(averagebinposition,radialdata[1:,14])
    plt.xlabel(r'r/$\sigma$')
    plt.ylabel('Percentage pointing inwards [%]')
    plt.axvline(x=2*boundaryradius*1e6,linewidth=0.5, color = 'k',linestyle='--')
    plt.axhline(y=50,linewidth=0.5, color = 'k',linestyle='--')
    ax6.axvline(x=2*R*1e6,linewidth=0.5, color = 'k',linestyle='--')
    plt.ylim([0.98*np.nanmin(radialdata[1:,14]),1.02*np.nanmax(radialdata[1:,14])])
    plt.xlim([0,2*boundaryradius*1e6])

    ax7 = plt.subplot(3,2,6)
    plt.plot(averagebinposition,radialdata[1:,13])
    plt.axhline(y=0,linewidth=0.5, color = 'k',linestyle='--')
    ax7.axvline(x=2*R*1e6,linewidth=0.5, color = 'k',linestyle='--')
    plt.xlabel(r'r/$\sigma$')
    plt.ylabel(r'$N_{in} \cdot v_{in} - N_{out} \cdot v_{out}$ [a.u.]')
    plt.xlim([0,2*boundaryradius*1e6])
    plt.axvline(x=boundaryradius*pxtomum,linewidth=0.5, color = 'k',linestyle='--')

    plt.suptitle('delay = '+str(round(delay*dt/reorientationtime,2))+r'$\cdot \tau_{r}$  -  $\tau_{r} = $'+str(round(reorientationtime,1))+' s')
    plt.tight_layout()  
    plt.savefig(string_temp+'Density layout'+str(name)+'.pdf', format='pdf')
    plt.close(fig)#plt.show()  
    #-------------------------------------
    if plothisto == True: 
        factor = 2.04
        smoothvalue = 20
        smoothit = smooth(velocity,smoothvalue)
        xmin = -factor*boundaryradius*1e6#x.min()
        xmax = factor*boundaryradius*1e6#x.max()
        ymin = -factor*boundaryradius*1e6#y.min()
        ymax = factor*boundaryradius*1e6#y.max()
        x = particlesdisplay[smoothvalue:maxlength-smoothvalue,0,0]
        y = particlesdisplay[smoothvalue:maxlength-smoothvalue,0,1]
        v = smoothit[smoothvalue:maxlength-smoothvalue]
        gridsizevalue = 30
        string_temp = savefolder + '/PlotHistogram/'
        if not os.path.exists(string_temp): os.makedirs(string_temp) 
        fig, axs = plt.subplots(ncols=2, figsize=(17,7), sharex='all')
        fig.subplots_adjust(hspace=0.1, left=0.05, right=0.95)
        ax = axs[0]
        circle2 = plt.Circle((0, 0), boundaryradius*pxtomum, color='k', fill=False)
        ax.add_artist(circle2)
        im = ax.scatter(x,y,c=v,linewidths=0.5)
        ax.set_xlabel('x [µm]')
        ax.set_ylabel('y [µm]')
        ax.tick_params(axis='y')
        ax.axis('equal')
        ax.set_xlim(xmin,xmax)
        ax.set_ylim(ymin,ymax)
        cb = fig.colorbar(im, ax=ax)
        cb.set_label('v in desired direction [µm/s]')
        x = particlesdisplay[:,0,0]
        y = particlesdisplay[:,0,1]
        #https://matplotlib.org/2.0.0/examples/pylab_examples/hexbin_demo.html
        ax = axs[1]
        hb = ax.hexbin(x, y, gridsize=gridsizevalue,mincnt=0.01)
        ax.axis([xmin, xmax, ymin, ymax])
        circle2 = plt.Circle((0, 0), boundaryradius*pxtomum, color='w', fill=False)
        ax.add_artist(circle2)
        cb = fig.colorbar(hb, ax=ax)
        cb.set_label('counts')
        ax.set_xlabel('x [µm]')
        ax.set_ylabel('y [µm]')
        ax.axis('equal')
        ax.set_xlim(xmin,xmax)
        ax.set_ylim(ymin,ymax)
        plt.tight_layout() 
        plt.savefig(string_temp+'Histogram'+str(name)+'.pdf', format='pdf')
        plt.close(fig)#plt.show() 
    #---------------------------------------
    radialdistance = np.zeros(maxlength)
    radialdistance[:] = np.sqrt(pow(particlesdisplay[:,0,0],2)+pow(particlesdisplay[:,0,1],2))
    driftdelay = 1   
    diffv = np.gradient(velocitydist)/(distmum[1]-distmum[0])
    maxpos = np.argmax(diffv)
    maxdist = distmum[maxpos]
    
    radialdist = np.zeros((steps,3))       #0 - radius, 1 - effective radial drift, 2 - counts
    base=(2*boundaryradius-diameter)*1e6/(steps-1)      # 6 - v inwards, 7 - v inwards err, 8 - density, 9 - v difference, 10 - v diff err, 11 - drift, 12 - drift err
    for i in range(steps): radialdist[i,0] = diameter*1e6+i*base
    for i in range(1, steps):
        radialdist[i,1] = round((radialdist[i,0]+radialdist[i-1,0])/2,1)
        #radialdist[i,2] = round(radialdist[i,1]/maxdist,1)     
    angletocenter = np.zeros(maxlength)
    for i in range(maxlength):
        a = [-np.cos((particles[i,0,3])*np.pi/180),-np.sin((particles[i,0,3])*np.pi/180)]
        b = [particlesdisplay[i,0,0],particlesdisplay[i,0,1]]
        angletocenter[i] = np.arccos(np.dot(a,b)/(vectorlength(a)*vectorlength(b)))*180/np.pi     
    radialdrift = np.zeros(maxlength)
    radialstats = np.zeros((steps,5))   #0 - radial distance, 1 - counts inwards, 2 - drift inwards
                                         #3 - counts outwards, 4 - drift inwards  
    limitedradialstats = np.zeros((steps,5))   #0 - radial distance, 1 - counts inwards, 2 - drift inwards
                                         #3 - counts outwards, 4 - drift inwards
    radialstats[:,0] = limitedradialstats[:,0] = radialdist[:,1]     
    allradialdata = np.zeros((maxlength,5))      #0 - not used, 1 - shell, 2 - in or out, 3 - radial drift
                                                 #4 - angletocenter
    limitedradialdata = np.zeros((maxlength,5))  #0 - step, 1 - shell, 2 - in or out, 3 - radial drift
    
    checkangle = 45
    count = 0
    inouttest = []; limitedinout = []  
    point = max(driftdelay,2)
    for i in range(point,maxlength):
        for j in range(1,steps):
            if radialdistance[i] < radialdist[j,0] and radialdistance[i] > radialdist[j-1,0]:
                check = (radialdistance[i]-radialdistance[i-driftdelay])/dt
                radialdrift[i] = (radialdistance[i]-radialdistance[i-driftdelay])/dt
                allradialdata[i,1] = radialdist[j,1]
                allradialdata[i,3] = abs(radialdrift[i])
                allradialdata[i,4] = angletocenter[i]
                if check <= 0:
                    radialstats[j,1] += 1
                    radialstats[j,2] += abs(radialdrift[i])
                    inouttest.append('inwards')
                    allradialdata[i,2] = 0
                    if angletocenter[i] < checkangle:
                        limitedradialstats[j,1] += 1
                        limitedradialstats[j,2] += abs(radialdrift[i])
                        limitedradialdata[count,0] = i
                        limitedradialdata[count,1] = radialdist[j,1]
                        limitedradialdata[count,2] = 0
                        limitedradialdata[count,3] = abs(radialdrift[i])
                        limitedradialdata[i,4] = angletocenter[i]
                        limitedinout.append('inwards')
                        count += 1
                else:
                    radialstats[j,3] += 1
                    radialstats[j,4] += radialdrift[i]
                    allradialdata[i,2] = 1
                    inouttest.append('outwards')
                    if angletocenter[i] > 180 - checkangle:
                        limitedradialstats[j,3] += 1
                        limitedradialstats[j,4] += abs(radialdrift[i])
                        limitedradialdata[count,0] = i
                        limitedradialdata[count,1] = radialdist[j,1]
                        limitedradialdata[count,2] = 1
                        limitedradialdata[count,3] = abs(radialdrift[i])
                        limitedradialdata[i,4] = angletocenter[i]
                        limitedinout.append('outwards')
                        count += 1
                j=steps
        if allradialdata[i,1] == 0: allradialdata[i,2] = 888
    radialstats[:,2] /= radialstats[:,1]
    radialstats[:,4] /= radialstats[:,3]
    limitedradialstats[:,2] /= limitedradialstats[:,1]
    limitedradialstats[:,4] /= limitedradialstats[:,3]
    
    vin = np.zeros(steps);vout = np.zeros(steps); limitedvin = np.zeros(steps); limitedvout = np.zeros(steps)
    for i in range(1,steps):
        vin[i] = round(radialstats[i,2],2)
        vout[i] = round(radialstats[i,4],2)
        limitedvin[i] = round(limitedradialstats[i,2],2)
        limitedvout[i] = round(limitedradialstats[i,4],2)
        
    alldata1 = list(zip(arange(point,maxlength),allradialdata[point:,1],inouttest[point:],allradialdata[point:,3]))
    alldata2 = pd.DataFrame(data = alldata1, columns=['step', 'distance', 'direction', 'velocity'])
    alldata3 = alldata2.replace(888, 'outside')
    limitedalldata1 = list(zip(arange(point,count),limitedradialdata[point:count,1],limitedinout[point:count],limitedradialdata[point:count,3]))
    limitedalldata2 = pd.DataFrame(data = limitedalldata1, columns=['step', 'distance', 'direction', 'velocity'])
    limitedalldata3 = limitedalldata2.replace(888, 'outside')

    string_temp = savefolderplus + '/PlotVelocityInandOut/'
    if not os.path.exists(string_temp): os.makedirs(string_temp)
    fig=plt.figure(1, figsize = (10,5) )
    ax4 = plt.subplot(1,1,1)
    ax4 = sns.violinplot(x="distance", y="velocity", hue="direction", hue_order = ['inwards','outwards'], order = radialdist[1:,1],
                            data=limitedalldata3, palette="coolwarm", split=True, scale="count", cut=0, inner=None)#,bw=.2, scale_hue=False
    ax4.set_ylabel('radial velocity ('+str(checkangle)+'°) [µm/s]')
    ax4.set_xlabel(r'r [µm]')
    numwave=np.linspace(0,len(vin[1:]),len(limitedvin[1:]),endpoint=False)
    for i in range(1,len(vin[1:])):
        ax4.plot([numwave[i-1],numwave[i-1]],[limitedvin[i],limitedvout[i]],color='k',linewidth=2.5)
    ax4.scatter(numwave,limitedvin[1:],s=50,c='b')#,label = 'v in')
    ax4.scatter(numwave,limitedvout[1:],s=50,c='r')#,label = 'v out')
    ax4.set_ylim([0,min(max(velocity),16)])
    ax4.set_title('Acceptance angle = '+str(checkangle)+'° - percentage = '+str(round(count/maxlength*100,1))+'%',loc='left')
    plt.legend(loc='upper left')
    plt.tight_layout() 
    plt.savefig(string_temp+'Velocity in and out - limited'+str(int(noise))+' - '+str(int(delay))+'.pdf', format='pdf')
    plt.close(fig)#plt.show()
    
    string_temp = savefolderplus + '/DataComparerInAndOut/'                  #0 - dist     1 - velocity     2 - velo err 3 - probability  4 - drift       5 - drift err  6 - anglechange  7 - passing     8 - r N steps before moving out 10 - count moving out 10
    if not os.path.exists(string_temp): os.makedirs(string_temp)                                                                                                                                                         #9 - r N steps before moving in   11 - count moving in
    np.savetxt(string_temp + 'DataComparerInAndOut'+str(p)+'.txt', np.transpose([radialdist[1:,1],limitedvin[1:],limitedvout[1:]]),fmt='%1.6f')

    string_temp = './DataIntensity/'                         
    if not os.path.exists(string_temp): os.makedirs(string_temp)                    #0      1         2        3          4          5          6          7            8            9
    np.savetxt(string_temp + 'DataIntensity'+str(p)+'.txt', np.transpose([radialdist[1:,1],radialdist[1:,2],vin[1:],vout[1:],limitedvin[1:],limitedvout[1:],radialdata[1:,2]]),fmt='%1.6f') 
    #-------------  
    string_temp = savefolderplus + '/DataGeneral/'
    if not os.path.exists(string_temp): os.makedirs(string_temp)           #0  1              2     3    4    5         6    7    8  9    10    
    np.savetxt(string_temp + 'DataGeneral'+str(p)+'.txt', np.transpose([dt,boundaryradius,delay,vmin,vmax,fitlength,sigma,D_0]),fmt='%1.6f')
    string_temp = './DataImportant/'                         
    if not os.path.exists(string_temp): os.makedirs(string_temp)                                                                                                                                                        
    np.savetxt(string_temp + 'Dataimportant'+str(p)+'.txt', np.transpose([meandistance*1e6,p,noise,delay,reorientationtime,meandensity,reorientationtime,maxvelocity[0],meanintensity,np.nanmean(velocity),timeclose,innerarea]),fmt='%1.6f')
    #0 mean dist center    1 # of paths, 2 mean anglechange, 3 count in, 4 noise, 5 delay, 6 reorientation time, 7 mean density, 8 reorientationtime2, 9/10 Drot/err, 11/12 Drotexp/err, 13/14 ptime/err, 15/16 plength/err, 17/18 more in/out, 19 vmax, 20 meantime, 21 meanprob, 22 notone, 23 zero, 24 mean velo    

    print("File: ",int(p),"  %s seconds " % round((time.time() - start_time),2),' - noise ',str(noise),' - delay ',str(delay))
    calctime += round((time.time() - start_time),2)
print('Total duration: ',calctime,' s')

/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:242: RuntimeWarning: Mean of empty slice
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:283: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:450: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:451: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:117: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:506: RuntimeWarni

File:  0   59.42 seconds   - noise  26  - delay  1


/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:242: RuntimeWarning: Mean of empty slice
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:283: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:450: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:451: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:117: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:506: RuntimeWarni

File:  1   59.73 seconds   - noise  26  - delay  5


/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:242: RuntimeWarning: Mean of empty slice
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:283: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:450: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:451: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:117: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:506: RuntimeWarni

...

File:  33   61.18 seconds   - noise  12  - delay  25


/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:242: RuntimeWarning: Mean of empty slice
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:283: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:450: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:451: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:117: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:506: RuntimeWarni

File:  34   62.12 seconds   - noise  12  - delay  29


/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:242: RuntimeWarning: Mean of empty slice
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:283: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:450: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:451: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:117: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: divide by zero encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:500: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:506: RuntimeWarni

File:  35   61.64 seconds   - noise  12  - delay  33
Total duration:  2214.8599999999997  s
